In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
data=pd.read_csv("../input/creditcard.csv")
data.head(10)

In [ ]:
print(data.shape)
print(data.Time.max())
print(data.Time.min())
print(data.Amount.max())
print(data.Amount.min())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data.iloc[:,[0,29]]=scaler.fit_transform(data.iloc[:,[0,29]])

In [ ]:
data.head()

In [ ]:
print(data.Amount.max())
print(data.Amount.min())

In [ ]:
class_count=pd.value_counts(data.Class,sort=True)

In [ ]:
class_count.plot(kind='bar')
plt.xlabel("Class")
plt.ylabel("Freq")
plt.title("Class vs Freq")

In [ ]:
X=data.ix[:,data.columns!="Class"]
Y=data.ix[:,data.columns=="Class"]
print(X.shape)
print(Y.shape)

# Resampling the data

In [ ]:
number_fraud=len(data[data.Class==1])
number_fraud_index=np.array(data[data.Class==1].index)
number_normal=np.array(data[data.Class==0].index)

number_normal_index=np.random.choice(number_normal,number_fraud,replace=False)
assert(number_normal_index.shape==number_fraud_index.shape)

In [ ]:
print(len(number_normal_index))

In [ ]:
undersample_index=np.concatenate([number_fraud_index,number_normal_index])
print(undersample_index.shape)

In [ ]:
undersample_data=data.iloc[undersample_index,:]
X_sampling=undersample_data.iloc[:,:30]
Y_sampling=undersample_data.iloc[:,30:]
print(X.shape)
print(Y.shape)

In [ ]:
print(len(undersample_data[undersample_data.Class==0]))

In [ ]:
from sklearn.model_selection import train_test_split
x_sampling_train,x_sampling_test,y_sampling_train,y_sampling_test=train_test_split(X_sampling,Y_sampling,test_size=0.3)

In [ ]:
print(x_sampling_train.shape)
print(x_sampling_test.shape)

In [ ]:
from sklearn.svm import SVC
classifier=SVC(kernel="rbf",probability=True)
classifier.fit(x_sampling_train,y_sampling_train)
y_sampling_pred=classifier.predict(x_sampling_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_sampling_test,y_sampling_pred)

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_sampling_test,y_sampling_pred)

# Testing on Whole Dataset

In [ ]:
Y_pred=classifier.predict(X)
print(confusion_matrix(Y,Y_pred))
print(recall_score(Y,Y_pred))

# Setting the threshold Probablity 

In [ ]:
threshold=[0.1,0.2,0.4,0.5]
for i in threshold:
    Y_pred_temp=classifier.predict_proba(X)
    Y_pred_temp=Y_pred_temp[:,1]>i
    print("-------------------------------------------------------------")
    print("Threshold : "+ str(i)+"\n")
    print(confusion_matrix(Y,Y_pred_temp))
    print("\n")
    print(recall_score(Y,Y_pred_temp))